# Experiment with Fashion MNIST
## Federated learning with random selection 3

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['AUTHGRAPH_VERBOSITY'] = '0'
import pickle
import time
import datetime
import random
import operator
import copy

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)
tf.random.set_seed(6292)
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
import tensorflow_hub as hub
import numpy as np
import pandas as pd

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
today = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
today

In [ ]:
labelnames = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
labelnames

In [ ]:
dataset_root = os.path.abspath(os.path.expanduser('dataset-fashionmnist'))
dataset_root

In [ ]:
model_root = os.path.abspath(os.path.expanduser(f'model-fashionmnist-random3-{today}'))
model_root

In [ ]:
found = pd.read_csv('round-fashionmnist.csv', index_col='Clients')
found.fillna(-1, inplace=True)
found = found.astype('int')
found.drop(columns=['Total'], inplace=True)
found.drop(index=['Total'], inplace=True)
found

In [ ]:
found_round = np.unique(found.values).tolist()
found_round.remove(-1)
found_round.sort(reverse=False)
found_round

In [ ]:
found_label = dict()
for fr in found_round:
    label = found_label.get('all', dict())
    label[fr] = found.columns[((-1 < found.loc[:, :]) & 
                               (found.loc[:, :] <= fr)).any()].tolist()
    found_label['all'] = label
    for c in found.index:
        label = found_label.get(c, dict())
        label[fr] = found.columns[((-1 < found.loc[c, :]) & 
                                   (found.loc[c, :] <= fr))].tolist()
        found_label[c] = label
found_label

In [ ]:
found_client = dict()
for fr in found_round:
    found_client[fr] = found.index[((-1 < found.loc[:, :]) & 
                                    (found.loc[:, :] <= fr)).any(axis=1)].tolist()
found_client